### Install enoslib (+ small setup)

In [ ]:
#!pip install -U pip
#!pip install enoslib
#!pip install python-grid5000
# need for credentials in .env
#!echo -e "username: $(head -n 1 .env)\npassword: $(head -n 2 .env | tail -n 1)" > ~/.python-grid5000.yaml

In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
print(conf_file)
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

### Launch machines

In [ ]:
job_name = 'dev'

cluster  = 'uvb'
conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        key="/home/adrien/.ssh/no_pass.pub",
        walltime="2:00:00"
    )    
    .add_machine(roles=["worker"], cluster=cluster, nodes=2)
)

provider = en.G5k(conf)

roles, networks = provider.init()

In [ ]:
provider.jobs

### mount local files on grid5000 

### Setup the machines

In [ ]:
with en.actions(roles=roles) as p:
    p.apt(name=["librdmacm1", "ibverbs-utils", "librdmacm-dev"], state="present")

### Kill every last one of them

In [ ]:
# Release all Grid'5000 resources
provider.destroy()